# NBA DFS Backtest Runner

Walk-forward backtesting for NBA DFS projections with flexible feature engineering.

**Data Source**: SQLite database at `nba_dfs.db`

In [1]:
import sys
import pandas as pd
import numpy as np
import logging
from pathlib import Path

sys.path.append('..')

from src.evaluation.backtest_config import BacktestConfig
from src.evaluation.walk_forward import WalkForwardBacktest
from src.evaluation.analysis import generate_backtest_report

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print('=' * 80)
print('NBA DFS BACKTEST NOTEBOOK')
print('=' * 80)
print('Initialized successfully')
print('=' * 80)

NBA DFS BACKTEST NOTEBOOK
Initialized successfully


## Experiment 1: Single Window

Baseline with 4-game rolling window.

In [ ]:
config_standard = BacktestConfig(
    start_date='20250102',
    end_date='20250110',
    lookback_days=60,
    model_type='xgboost',
    rolling_window_sizes=[3,10]
)

print(f"Rolling Windows: {config_standard.rolling_window_sizes}")
print(f"Features: {len(config_standard.features_to_use)}")

backtest = WalkForwardBacktest(config_standard, db_path='../nba_dfs.db')
results_standard = backtest.run()

if 'error' not in results_standard:
    print(f"\n{'='*80}")
    print("RESULTS - STANDARD FEATURES")
    print(f"{'='*80}")
    print(f"MAPE: {results_standard['mean_mape']:.1f}%")
    print(f"RMSE: {results_standard['mean_rmse']:.2f}")
    print(f"Correlation: {results_standard['mean_correlation']:.3f}")
    print(f"Slates: {results_standard['num_slates']}")
else:
    print(f"ERROR: {results_standard['error']}")

## Experiment 2: Dual Window

Window-based features with 4 and 10-game windows.

In [ ]:
config_rolling = BacktestConfig(
    start_date='20231101',
    end_date='20231130',
    lookback_days=90,
    model_type='xgboost',
    rolling_window_sizes=[4, 10]
)

print(f"Rolling Windows: {config_rolling.rolling_window_sizes}")
print(f"Features: {len(config_rolling.features_to_use)}")

backtest = WalkForwardBacktest(config_rolling, db_path='../nba_dfs.db')
results_rolling = backtest.run()

if 'error' not in results_rolling:
    print(f"\n{'='*80}")
    print("RESULTS - ROLLING WINDOW FEATURES (4g)")
    print(f"{'='*80}")
    print(f"MAPE: {results_rolling['mean_mape']:.1f}%")
    print(f"RMSE: {results_rolling['mean_rmse']:.2f}")
    print(f"Correlation: {results_rolling['mean_correlation']:.3f}")
    print(f"Slates: {results_rolling['num_slates']}")
else:
    print(f"ERROR: {results_rolling['error']}")

## Experiment 3: Multiple Windows

Test multiple temporal perspectives: [3, 4, 5, 10] games.

In [ ]:
config_multi = BacktestConfig(
    start_date='20231101',
    end_date='20231130',
    lookback_days=90,
    model_type='xgboost',
    rolling_window_sizes=[3, 4, 5, 10]
)

print(f"Rolling Windows: {config_multi.rolling_window_sizes}")
print(f"Features: {len(config_multi.features_to_use)}")

backtest = WalkForwardBacktest(config_multi, db_path='../nba_dfs.db')
results_multi = backtest.run()

if 'error' not in results_multi:
    print(f"\n{'='*80}")
    print("RESULTS - MULTIPLE WINDOWS [3,4,5,10]")
    print(f"{'='*80}")
    print(f"MAPE: {results_multi['mean_mape']:.1f}%")
    print(f"RMSE: {results_multi['mean_rmse']:.2f}")
    print(f"Correlation: {results_multi['mean_correlation']:.3f}")
    print(f"Slates: {results_multi['num_slates']}")
else:
    print(f"ERROR: {results_multi['error']}")

## Experiment 4: Single Window Baseline

Single 4-game window as baseline comparison.

In [ ]:
config_combined = BacktestConfig(
    start_date='20231101',
    end_date='20231130',
    lookback_days=90,
    model_type='xgboost',
    rolling_window_sizes=[4]
)

print(f"Rolling Windows: {config_combined.rolling_window_sizes}")
print(f"Total Features: {len(config_combined.features_to_use)}")

backtest = WalkForwardBacktest(config_combined, db_path='../nba_dfs.db')
results_combined = backtest.run()

if 'error' not in results_combined:
    print(f"\n{'='*80}")
    print("RESULTS - COMBINED FEATURES")
    print(f"{'='*80}")
    print(f"MAPE: {results_combined['mean_mape']:.1f}%")
    print(f"RMSE: {results_combined['mean_rmse']:.2f}")
    print(f"Correlation: {results_combined['mean_correlation']:.3f}")
    print(f"Slates: {results_combined['num_slates']}")
else:
    print(f"ERROR: {results_combined['error']}")

## Comparison

Compare all feature sets.

In [ ]:
comparison_results = []

if 'error' not in results_standard:
    comparison_results.append({
        'name': 'Standard',
        'mape': results_standard['mean_mape'],
        'rmse': results_standard['mean_rmse'],
        'correlation': results_standard['mean_correlation']
    })

if 'error' not in results_rolling:
    comparison_results.append({
        'name': 'Rolling (4g)',
        'mape': results_rolling['mean_mape'],
        'rmse': results_rolling['mean_rmse'],
        'correlation': results_rolling['mean_correlation']
    })

if 'error' not in results_multi:
    comparison_results.append({
        'name': 'Multi-Window',
        'mape': results_multi['mean_mape'],
        'rmse': results_multi['mean_rmse'],
        'correlation': results_multi['mean_correlation']
    })

if 'error' not in results_combined:
    comparison_results.append({
        'name': 'Combined',
        'mape': results_combined['mean_mape'],
        'rmse': results_combined['mean_rmse'],
        'correlation': results_combined['mean_correlation']
    })

comparison_df = pd.DataFrame(comparison_results)
print(f"\n{'='*80}")
print("FEATURE SET COMPARISON")
print(f"{'='*80}")
print(comparison_df.to_string(index=False))

## Visualization

Compare MAPE across feature sets.

In [ ]:
import matplotlib.pyplot as plt

if len(comparison_results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    names = [r['name'] for r in comparison_results]
    mapes = [r['mape'] for r in comparison_results]
    rmses = [r['rmse'] for r in comparison_results]
    
    axes[0].bar(names, mapes, color=['#2E86AB', '#A23B72', '#F18F01', '#06A77D'])
    axes[0].axhline(y=30, color='red', linestyle='--', linewidth=2, label='30% Target')
    axes[0].set_ylabel('MAPE (%)', fontsize=12, fontweight='bold')
    axes[0].set_title('Prediction Accuracy by Feature Set', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    axes[1].bar(names, rmses, color=['#2E86AB', '#A23B72', '#F18F01', '#06A77D'])
    axes[1].set_ylabel('RMSE (points)', fontsize=12, fontweight='bold')
    axes[1].set_title('Prediction Error by Feature Set', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    best_idx = np.argmin(mapes)
    print(f"\nBest Feature Set: {names[best_idx]}")
    print(f"MAPE: {mapes[best_idx]:.1f}%")
    print(f"RMSE: {rmses[best_idx]:.2f}")
else:
    print("No comparison results to plot")

## Summary

Results and next steps.

In [ ]:
print(f"{'='*80}")
print("BACKTEST SUMMARY")
print(f"{'='*80}\n")

print("Experiments Run:")
print("1. Single Window [4] - 4-game rolling window")
print("2. Dual Window [4,10] - Short and long window features")
print("3. Multiple Windows [3,4,5,10] - Multi-temporal perspectives")
print("4. Single Window Baseline - 4-game window only\n")

print("Next Steps:")
print("1. Run full season backtest for production validation")
print("2. Add DvP and Vegas line features")
print("3. Implement lineup optimization")
print("4. Check feature importance\n")

print(f"{'='*80}")